# ImgLib2 

Example 01: Opening, creating and displaying images

Let's get started with ImgLib2 by opening, creating, and displaying images.  You can see these same tutorials on the ImageJ Wiki [here](http://imagej.net/ImgLib2_Examples).

In [ ]:
// Behind a firewall? Configure your proxy settings here.
//systemProperties = System.getProperties()
//systemProperties.setProperty("http.proxyHost","myproxy.domain")
//systemProperties.setProperty("http.proxyPort","8080")

//////////////////////////////////////////////////////////////
// Load ImageJ. This may take some minutes the first time   //
// while ImageJ is installed into ~/.groovy/grapes locally. //
//////////////////////////////////////////////////////////////
@GrabResolver(name='imagej', root='http://maven.imagej.net/content/groups/public/')
@Grab('net.imagej:imagej:2.0.0-rc-58')
import net.imagej.ImageJ
ij = new ImageJ()
println("ImageJ is ready to go.")

## Opening an ImgLib2 image

The typical way to open an image in ImgLib2 is to make use of the [SCIFIO](http://scif.io/) importer. Below you see two examples of how to open an image as (_a_) its own type (e.g. **UnsignedByteType**) and (_b_) as float (**FloatType**). For (_a_) we assume, however, that the file contains some real valued numbers as defined by the interface **RealType**. Color images are opened as well and color is represented as its own dimension (like in the ImageJ Hyperstacks).

Important: it does not matter which type of Img you use to hold the data as we will use [Iterator](http://javadoc.imagej.net/ImgLib2/net/imglib2/Iterator.html)s and [RandomAccess](http://javadoc.imagej.net/ImgLib2/net/imglib2/RandomAccess.html)es to access the image content. It might be, however, important if you work on two Img at the same time using [Iterator](http://javadoc.imagej.net/ImgLib2/net/imglib2/Iterator.html)s, see Example2.

In the following example,  (_a_), we use an [ArrayImg](http://javadoc.imagej.net/ImgLib2/net/imglib2/img/array/ArrayImg.html) to hold the data. This means the data is held in one single java basic type array which results in optimal performance. The absolute size of image is, however, limited to 2^31-1 (~2 billion) pixels. The type of [Img](http://javadoc.imagej.net/ImgLib2/net/imglib2/img/Img.html) to use is set by passing an **ImgOptions** configuration when calling the **ImgOpener**.

In [ ]:
import io.scif.img.ImgOpener

// define the file to open
path = "http://samples.fiji.sc/tutorials/DrosophilaWing.tif"
 
// create the ImgOpener. The type (e.g. ArrayImg, PlanarImg, CellImg) is
// automatically determined. For a small image that fits in memory, this
// should open as an ArrayImg.
imgOpener = new ImgOpener()
 
// open with ImgOpener. 
image = imgOpener.openImgs(path).get(0)
 
// show image
ij.notebook().display( image )

In (_b_) just below, we use a [CellImg](http://javadoc.imagej.net/ImgLib2/net/imglib2/img/cell/CellImg.html) instead. It partitions the image data into n-dimensional cells each holding only a part of the data. Further, [SCIFIO](http://scif.io/) takes care of caching cells in and out of memory as needed, greatly reducing the memory requirement to work with very large images.

The SCIFIO importer also requires [Type](http://javadoc.imagej.net/ImgLib2/net/imglib2/type/Type.html)s that implement [NativeType](http://javadoc.imagej.net/ImgLib2/net/imglib2/type/NativeType.html), which means it is able to map the data into a Java basic type array. All available [Type](http://javadoc.imagej.net/ImgLib2/net/imglib2/type/Type.html)s until now are implementing [NativeType](http://javadoc.imagej.net/ImgLib2/net/imglib2/type/NativeType.html), if you want to work with some self-developed [Type](http://javadoc.imagej.net/ImgLib2/net/imglib2/type/Type.html) it would be easiest to copy the opened [Img](http://javadoc.imagej.net/ImgLib2/net/imglib2/img/Img.html) afterwards. Please also note that until now, the only [Img](http://javadoc.imagej.net/ImgLib2/net/imglib2/img/Img.html) that supports non-native types is the [ListImg](http://javadoc.imagej.net/ImgLib2/net/imglib2/img/list/ListImg.html) which stores every pixel as an individual object!

In [ ]:
import io.scif.config.SCIFIOConfig
import io.scif.config.SCIFIOConfig.ImgMode

// create the SCIFIOConfig. This gives us configuration control over how
// the ImgOpener will open its datasets.
config = new SCIFIOConfig();
 
// If we know what type of Img we want, we can encourage their use through
// an SCIFIOConfig instance. CellImgs dynamically load image regions and are
// useful when an image won't fit in memory
config.imgOpenerSetImgModes( ImgMode.CELL );
 
// open with ImgOpener as a CellImg
imageCell = imgOpener.openImg( path, config );
 
// show image
ij.notebook().display( imageCell )

## Creating a new ImgLib2 image

Another important way to instantiate a new ImgLib2 [Img](http://javadoc.imagej.net/ImgLib2/net/imglib2/img/Img.html) is to create a new one from scratch. This requires you to define its [Type](http://javadoc.imagej.net/ImgLib2/net/imglib2/type/Type.html) as well as the [ImgFactory](http://javadoc.imagej.net/ImgLib2/net/imglib2/img/ImgFactory.html) to use. It does additionally need one instance of the [Type](http://javadoc.imagej.net/ImgLib2/net/imglib2/type/Type.html) that it is supposed to hold.

In [ ]:
import net.imglib2.img.cell.CellImgFactory
import net.imglib2.type.numeric.real.FloatType

// create the ImgFactory based on cells (cellsize = 5x5x5...x5) that will
// instantiate the Img
imgFactory = new CellImgFactory< FloatType >( 5 )
 
// create an 3d-Img with dimensions 20x30x40 (here cellsize is 5x5x5)Ø
long[] dimensions = [20, 30, 40]
img1 = imgFactory.create( dimensions, new FloatType() )
 
// display img1
ij.notebook().display( img1 )

Once you have one instance of an [Img](http://javadoc.imagej.net/ImgLib2/net/imglib2/img/Img.html), it is very easy to create another one using the same [Type](http://javadoc.imagej.net/ImgLib2/net/imglib2/type/Type.html) and [ImgFactory](http://javadoc.imagej.net/ImgLib2/net/imglib2/img/ImgFactory.html), even if it has a different size. Note that the call [img.firstElement()](http://javadoc.imagej.net/ImgLib2/net/imglib2/IterableRealInterval.html#firstElement--) returns the first pixel of any [Iterable](http://javadoc.imagej.net/Java/java/lang/Iterable.html?is-external=true), e.g. an [Img](http://javadoc.imagej.net/ImgLib2/net/imglib2/img/Img.html).

In [ ]:
import net.imglib2.img.Img;
import net.imglib2.img.ImgFactory;

// create another image with the same size
// note that the input provides the size for the new image as it implements
// the Interval interface
img2 = imgFactory.create( img1, img1.firstElement() )

// display img2
ij.notebook().display( img2 ) 

## Displaying images partly using Views

By using the concept of [Views](http://javadoc.imagej.net/ImgLib2/net/imglib2/view/Views.html) it is possible to display only parts of the image, display a rotated view, and many more cool things. Note that you can also concatenate them. [Views](http://javadoc.imagej.net/ImgLib2/net/imglib2/view/Views.html) are much more powerful than shown in this example, they will be increasingly used throughout the examples.

A [View](http://javadoc.imagej.net/ImgLib2/net/imglib2/View.html) almost behaves similar to an [Img](http://javadoc.imagej.net/ImgLib2/net/imglib2/img/Img.html), and in fact they share important concepts. Both are [RandomAccessible](http://javadoc.imagej.net/ImgLib2/net/imglib2/RandomAccessible.html), and [Views](http://javadoc.imagej.net/ImgLib2/net/imglib2/view/Views.html) that are not infinite are also an [Interval](http://javadoc.imagej.net/ImgLib2/net/imglib2/Interval.html) (i.e. those [Views](http://javadoc.imagej.net/ImgLib2/net/imglib2/view/Views.html) have a defined size) and can therefore be made [Iterable](http://javadoc.imagej.net/Java/java/lang/Iterable.html?is-external=true) (see example 2c). In ImgLib2, all algorithms are implemented for abstract concepts like [RandomAccessible](http://javadoc.imagej.net/ImgLib2/net/imglib2/RandomAccessible.html), [Iterable](http://javadoc.imagej.net/Java/java/lang/Iterable.html?is-external=true) or [Interval](http://javadoc.imagej.net/ImgLib2/net/imglib2/Interval.html). This enables us, as can be seen below, to display a [View](http://javadoc.imagej.net/ImgLib2/net/imglib2/View.html) the exact same way we would also display an [Img](http://javadoc.imagej.net/ImgLib2/net/imglib2/img/Img.html).

In [ ]:
import io.scif.img.ImgOpener
import net.imglib2.type.numeric.real.FloatType
import net.imglib2.view.Views

// open file as float with ImgOpener
img = new ImgOpener().openImg( "http://samples.fiji.sc/tutorials/DrosophilaWing.tif", new FloatType() )
 
// use a View to define an interval (min and max coordinate, inclusive) to display
long[] min = [200, 200]
long[] max = [500, 350]
view = Views.interval( img, min, max)
 
// display only the part of the Img
ij.notebook().display( view )

Now, we will display the partial image at a 90&deg rotation.

In [ ]:
import net.imglib2.view.Views

// this is shifting the image so that it is back 
// in alignment with origin for notebook displaying purposes
viewRotated = Views.rotate(view, 0, 1) 

// display the same area rotated by 90 degrees (x-axis (0) and y-axis (1) switched)
ij.notebook().display( viewRotated )